In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
from six.moves import cPickle
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency, metrics

import helper

np.random.seed(247)
tf.set_random_seed(247)

In [2]:
data_path = '../data/Synthetic_dataset.h5'
results_path = '../results'
params_path = utils.make_directory(results_path, 'model_params')

# load dataset
train, valid, test = helper.load_synthetic_dataset(data_path)

test_model = helper.load_synthetic_models(data_path, dataset='test')
    
# get data shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = [None, train['targets'].shape[1]]

loading training data
loading cross-validation data
loading test data


# guided backprop

In [3]:
true_index = np.where(test['targets'][:,0] == 1)[0]
X = test['inputs'][true_index]
X_model = test_model[true_index]

dropout_status = [True, True, False, False, False, True, True, False]
l2_status = [True, False, True, False, False, True, False, True]
bn_status = [True, False, False, True, False, False, True, True]

all_models = ['StandardNet', 
              'DeepBind', 
              'DistNet', 
              'LocalNet']

guided_results ={}
for model_name in all_models:

    # model save path
    model_path = utils.make_directory(params_path, model_name)
    
    for i in range(len(dropout_status)):

        name = model_name
        if dropout_status[i]:
            name += '_do'
        if l2_status[i]:
            name += '_l2'
        if bn_status[i]:
            name += '_bn'
        
        file_path = os.path.join(model_path, name)

        # saliency parameters
        params = {'model_name': model_name, 
                  'input_shape': input_shape, 
                  'dropout_status': dropout_status[i],
                  'l2_status': l2_status[i],
                  'bn_status': bn_status[i],
                  'model_path': file_path+'_best.ckpt',
                 }

        guided_saliency = helper.backprop(X, params, layer='output', class_index=None, method='guided')

        info = []
        for j, gs in enumerate(guided_saliency):
            X_saliency = np.squeeze(gs).T
            good_info, bad_info = helper.entropy_weighted_cosine_distance(X_saliency, X_model[j])
            info.append([tpr, fpr])
        info = np.array(info)
        
        guided_results[name] = info
        
with open(os.path.join(save_path, 'guided_backprop.pickle'), 'wb') as f:
    cPickle.dump(guided_results, f, protocol=cPickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/DeepNet/DeepNet_do_l2_bn_best.ckpt


# backprop

In [ ]:
true_index = np.where(test['targets'][:,0] == 1)[0]
X = test['inputs'][true_index]
X_model = test_model[true_index]

dropout_status = [True, True, False, False, False, True, True, False]
l2_status = [True, False, True, False, False, True, False, True]
bn_status = [True, False, False, True, False, False, True, True]

all_models = ['StandardNet', 
              'DeepBind', 
              'DistNet', 
              'LocalNet']

backprop_results ={}
for model_name in all_models:

    # model save path
    model_path = utils.make_directory(params_path, model_name)
    
    for i in range(len(dropout_status)):

        name = model_name
        if dropout_status[i]:
            name += '_do'
        if l2_status[i]:
            name += '_l2'
        if bn_status[i]:
            name += '_bn'
        
        file_path = os.path.join(model_path, name)

        # saliency parameters
        params = {'model_name': model_name, 
                  'input_shape': input_shape, 
                  'dropout_status': dropout_status[i],
                  'l2_status': l2_status[i],
                  'bn_status': bn_status[i],
                  'model_path': file_path+'_best.ckpt',
                 }

        guided_saliency = helper.backprop(X, params, layer='output', class_index=None, method='backprop')

        info = []
        for j, gs in enumerate(guided_saliency):
            X_saliency = np.squeeze(gs).T
            tpr, fpr = helper.entropy_weighted_cosine_distance(X_saliency, X_model[j])
            info.append([tpr, fpr])
        info = np.array(info)
        
        backprop_results[name] = info
    
with open(os.path.join(save_path, 'backprop.pickle'), 'wb') as f:
    cPickle.dump(backprop_results, f, protocol=cPickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_do_l2_bn_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_do_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_l2_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_bn_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_do_l2_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_do_bn_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/regulatory_code/model_params/StandardNet/StandardNet_l2_bn_best.ckpt
INFO:tensorflow:Restoring parameters